In [1]:
# %store -r

In [2]:
from glob import glob
from os import getcwd, makedirs
import os
from pathlib import Path
import re

In [3]:
import pandas as pd

In [4]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_colwidth', 1000)

In [5]:
df = pd.read_table('analysis/HIP00769.tsv', low_memory=False)

In [6]:
df.sample(1)

,rearrangement,amino_acid,frame_type,rearrangement_type,templates,reads,frequency,productive_frequency,cdr3_length,v_family,v_gene,v_allele,d_family,d_gene,d_allele,j_family,j_gene,j_allele,v_deletions,d5_deletions,d3_deletions,j_deletions,n2_insertions,n1_insertions,v_index,n1_index,n2_index,d_index,j_index,v_family_ties,v_gene_ties,v_allele_ties,d_family_ties,d_gene_ties,d_allele_ties,j_family_ties,j_gene_ties,j_allele_ties,sequence_tags,v_shm_count,v_shm_indexes,antibody,sample_name,species,locus,product_subtype,kit_pool,total_templates,productive_templates,outofframe_templates,stop_templates,dj_templates,total_rearrangements,productive_rearrangements,outofframe_rearrangements,stop_rearrangements,dj_rearrangements,total_reads,total_productive_reads,total_outofframe_reads,total_stop_reads,total_dj_reads,productive_clonality,productive_entropy,sample_clonality,sample_entropy,sample_amount_ng,sample_cells_mass_estimate,fraction_productive_of_cells_mass_estimate,sample_cells,fraction_productive_of_cells,max_productive_frequency,max_frequency,counting_method,primer_set,release_date,sample_tags,fraction_productive,order_name,kit_id,total_t_cells
168692,GTGACCAGTGCCCATCCTGAAGACAGCAGCTTCTACATCTGCAGTGCCCTCCCAGCGGGGGATATAGATACGCAGTATTTTGGCCCA,CSALPAGDIDTQYF,In,VDJ,1,8,0.000002,0.000002,42,TCRBV20,unresolved,NaN,TCRBD02,TCRBD02-01,1.0,TCRBJ02,TCRBJ02-03,1.0,6,6,2,5,4,6,39,47,61,53,65,NaN,"TCRBV20-01,TCRBV20-or09_02",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HIP00769,Human,TCRB,Deep,NaN,293596,250035,40135,3424,0,238171,203018,32160,2993,0,5245352,4467114,717050,61188,0,0.042246,16.886391,0.044277,17.070774,1099.64001,169175,1.477967,0,0.0,0.005042,0.005439,v2,Human-TCRB-PD1x,2013-12-13 22:22:21.368,"Age:23 Years,Biological Sex:Female,Cohort:Cohort 01,Ethnic Group:Unknown Ethnicity,HLA MHC class I:HLA-A*01,HLA MHC class I:HLA-A*32,HLA MHC class I:HLA-B*08,HLA MHC class I:HLA-B*44,Inferred CMV status (cross-validation): Inferred CMV -,Inferred CMV status: Inferred CMV -,Inferred HLA type:Inferred HLA-A*01,Inferred HLA type:Inferred HLA-A*32,Inferred HLA type:Inferred HLA-B*08,Inferred HLA type:Inferred HLA-B*44,Racial Group:Unknown racial group,Species:Human,Tissue Source:gDNA,Tissue Source:PBMC,Tissue Source:Peripheral blood lymphocytes (PBL),Tissue Source:T cells,Virus Diseases:Cytomegalovirus -",0.851629,NaN,NaN,0


In [7]:
PATH = r'/home/vova/Documents/Projects/Receptor/emerson_2017/'
files = glob(PATH + '*')
files

['/home/vova/Documents/Projects/Receptor/emerson_2017/HIP13703.tsv',
 '/home/vova/Documents/Projects/Receptor/emerson_2017/HIP12123.tsv',
 '/home/vova/Documents/Projects/Receptor/emerson_2017/HIP13966.tsv',
 '/home/vova/Documents/Projects/Receptor/emerson_2017/HIP08337.tsv',
 '/home/vova/Documents/Projects/Receptor/emerson_2017/HIP08389.tsv',
 '/home/vova/Documents/Projects/Receptor/emerson_2017/HIP00640.tsv',
 '/home/vova/Documents/Projects/Receptor/emerson_2017/HIP13325.tsv',
 '/home/vova/Documents/Projects/Receptor/emerson_2017/Keck0023_MC1.tsv',
 '/home/vova/Documents/Projects/Receptor/emerson_2017/HIP14183.tsv',
 '/home/vova/Documents/Projects/Receptor/emerson_2017/HIP00822.tsv',
 '/home/vova/Documents/Projects/Receptor/emerson_2017/HIP13663.tsv',
 '/home/vova/Documents/Projects/Receptor/emerson_2017/HIP05763.tsv',
 '/home/vova/Documents/Projects/Receptor/emerson_2017/HIP05331.tsv',
 '/home/vova/Documents/Projects/Receptor/emerson_2017/HIP13695.tsv',
 '/home/vova/Documents/Project

In [8]:
def extract_info(input_path, output_path, columns):
    print(input_path)
    
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    
    name = Path(input_path).stem
    
    print(name)

    df = pd.read_table(input_path, usecols=columns)
    
    print('Before First Filter', df.shape)
    df.dropna(axis=0, inplace=True)
    print('After First Filter', df.shape)
        
    mask = df.amino_acid.str.contains('*', regex=False)
    df = df[~mask]
    df.reset_index(drop=True, inplace=True)

    print('After Second Filter', df.shape)
    
    cols = ['amino_acid', 'v_family', 'j_family']
    df['combined'] = df[cols].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)
    
    try:
        target = re.search('(?<=Cytomegalovirus\s).{1}', df.sample_tags[0])[0]
    
        df['target'] = target
        
        df.drop('sample_tags', axis=1, inplace=True)
    
        switch = {'-': 'negative', '+': 'positive'}
    
        new_path = ''.join([output_path, name, '_', switch[target], '.csv'])
        
        cols = ['amino_acid', 'v_family', 'j_family']
        df['combined'] = df[cols].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)
    
        df.to_csv(new_path, index=False)
    
    except Exception as e:
        print(e)
        print('\nThere is no target in sample_tags columns\n')

In [9]:
%%time
output_path = ''.join([getcwd(), '/PreprocessData/'])    


columns = ['sample_tags', 'v_family', 'j_family', 'amino_acid', 'sample_name']

for file in files:
    extract_info(file, output_path, columns)

/home/vova/Documents/Projects/Receptor/emerson_2017/HIP13703.tsv
HIP13703
Before First Filter (101931, 5)
After First Filter (84871, 5)
After Second Filter (83370, 5)
/home/vova/Documents/Projects/Receptor/emerson_2017/HIP12123.tsv
HIP12123
Before First Filter (231011, 5)
After First Filter (195214, 5)
After Second Filter (191455, 5)
/home/vova/Documents/Projects/Receptor/emerson_2017/HIP13966.tsv
HIP13966
Before First Filter (259136, 5)
After First Filter (224192, 5)
After Second Filter (221281, 5)
/home/vova/Documents/Projects/Receptor/emerson_2017/HIP08337.tsv
HIP08337
Before First Filter (162946, 5)
After First Filter (135335, 5)
After Second Filter (133029, 5)
/home/vova/Documents/Projects/Receptor/emerson_2017/HIP08389.tsv
HIP08389
Before First Filter (411496, 5)
After First Filter (343745, 5)
After Second Filter (337164, 5)
/home/vova/Documents/Projects/Receptor/emerson_2017/HIP00640.tsv
HIP00640
Before First Filter (253388, 5)
After First Filter (209453, 5)
After Second Filter 

Before First Filter (267769, 5)
After First Filter (215672, 5)
After Second Filter (211296, 5)
/home/vova/Documents/Projects/Receptor/emerson_2017/Keck0117_MC1.tsv
Keck0117_MC1
Before First Filter (331272, 5)
After First Filter (277232, 5)
After Second Filter (272394, 5)
/home/vova/Documents/Projects/Receptor/emerson_2017/HIP13489.tsv
HIP13489
Before First Filter (241742, 5)
After First Filter (204333, 5)
After Second Filter (200816, 5)
/home/vova/Documents/Projects/Receptor/emerson_2017/HIP01266.tsv
HIP01266
Before First Filter (339108, 5)
After First Filter (281242, 5)
After Second Filter (275668, 5)
/home/vova/Documents/Projects/Receptor/emerson_2017/HIP13497.tsv
HIP13497
Before First Filter (215789, 5)
After First Filter (186353, 5)
After Second Filter (183972, 5)
/home/vova/Documents/Projects/Receptor/emerson_2017/HIP13217.tsv
HIP13217
Before First Filter (182703, 5)
After First Filter (152196, 5)
After Second Filter (149650, 5)
/home/vova/Documents/Projects/Receptor/emerson_2017/

Before First Filter (173716, 5)
After First Filter (143234, 5)
After Second Filter (140443, 5)
/home/vova/Documents/Projects/Receptor/emerson_2017/HIP13720.tsv
HIP13720
Before First Filter (274544, 5)
After First Filter (231916, 5)
After Second Filter (227965, 5)
/home/vova/Documents/Projects/Receptor/emerson_2017/HIP13294.tsv
HIP13294
Before First Filter (279929, 5)
After First Filter (233117, 5)
After Second Filter (228507, 5)
/home/vova/Documents/Projects/Receptor/emerson_2017/HIP01004.tsv
HIP01004
Before First Filter (309878, 5)
After First Filter (259623, 5)
After Second Filter (255255, 5)
/home/vova/Documents/Projects/Receptor/emerson_2017/HIP13473.tsv
HIP13473
Before First Filter (398714, 5)
After First Filter (333813, 5)
After Second Filter (327477, 5)
/home/vova/Documents/Projects/Receptor/emerson_2017/HIP10597.tsv
HIP10597
Before First Filter (175567, 5)
After First Filter (149334, 5)
After Second Filter (146805, 5)
/home/vova/Documents/Projects/Receptor/emerson_2017/HIP13746

/home/vova/Documents/Projects/Receptor/emerson_2017/Keck0100_MC1.tsv
Keck0100_MC1
Before First Filter (222512, 5)
After First Filter (188257, 5)
After Second Filter (185380, 5)
/home/vova/Documents/Projects/Receptor/emerson_2017/HIP00728.tsv
HIP00728
Before First Filter (116888, 5)
After First Filter (95062, 5)
After Second Filter (93035, 5)
/home/vova/Documents/Projects/Receptor/emerson_2017/HIP08399.tsv
HIP08399
Before First Filter (293069, 5)
After First Filter (243254, 5)
After Second Filter (239088, 5)
/home/vova/Documents/Projects/Receptor/emerson_2017/HIP08888.tsv
HIP08888
Before First Filter (352263, 5)
After First Filter (296091, 5)
After Second Filter (290934, 5)
/home/vova/Documents/Projects/Receptor/emerson_2017/HIP08653.tsv
HIP08653
Before First Filter (319860, 5)
After First Filter (274613, 5)
After Second Filter (270694, 5)
/home/vova/Documents/Projects/Receptor/emerson_2017/Keck0052_MC1.tsv
Keck0052_MC1
Before First Filter (43443, 5)
After First Filter (35990, 5)
After 

/home/vova/Documents/Projects/Receptor/emerson_2017/HIP05838.tsv
HIP05838
Before First Filter (328357, 5)
After First Filter (275316, 5)
After Second Filter (270131, 5)
/home/vova/Documents/Projects/Receptor/emerson_2017/Keck0049_MC1.tsv
Keck0049_MC1
Before First Filter (251601, 5)
After First Filter (204883, 5)
After Second Filter (200062, 5)
/home/vova/Documents/Projects/Receptor/emerson_2017/HIP10514.tsv
HIP10514
Before First Filter (307470, 5)
After First Filter (259033, 5)
After Second Filter (254961, 5)
/home/vova/Documents/Projects/Receptor/emerson_2017/HIP13938.tsv
HIP13938
Before First Filter (133566, 5)
After First Filter (118881, 5)
After Second Filter (117644, 5)
/home/vova/Documents/Projects/Receptor/emerson_2017/HIP01805.tsv
HIP01805
Before First Filter (358816, 5)
After First Filter (292085, 5)
After Second Filter (285949, 5)
/home/vova/Documents/Projects/Receptor/emerson_2017/HIP13567.tsv
HIP13567
Before First Filter (260017, 5)
After First Filter (211870, 5)
After Seco

...

/home/vova/Documents/Projects/Receptor/emerson_2017/HIP13396.tsv
HIP13396
Before First Filter (421237, 5)
After First Filter (344536, 5)
After Second Filter (337404, 5)
/home/vova/Documents/Projects/Receptor/emerson_2017/HIP12980.tsv
HIP12980
Before First Filter (269490, 5)
After First Filter (230205, 5)
After Second Filter (226403, 5)
/home/vova/Documents/Projects/Receptor/emerson_2017/HIP02780.tsv
HIP02780
Before First Filter (180323, 5)
After First Filter (149083, 5)
After Second Filter (146117, 5)
/home/vova/Documents/Projects/Receptor/emerson_2017/HIP09029.tsv
HIP09029
Before First Filter (391825, 5)
After First Filter (329136, 5)
After Second Filter (323236, 5)
/home/vova/Documents/Projects/Receptor/emerson_2017/HIP13902.tsv
HIP13902
Before First Filter (232370, 5)
After First Filter (194701, 5)
After Second Filter (191202, 5)
/home/vova/Documents/Projects/Receptor/emerson_2017/Keck0103_MC1.tsv
Keck0103_MC1
Before First Filter (202802, 5)
After First Filter (168558, 5)
After Seco

Before First Filter (372382, 5)
After First Filter (297657, 5)
After Second Filter (290470, 5)
/home/vova/Documents/Projects/Receptor/emerson_2017/HIP11649.tsv
HIP11649
Before First Filter (244737, 5)
After First Filter (201396, 5)
After Second Filter (197020, 5)
/home/vova/Documents/Projects/Receptor/emerson_2017/HIP05551.tsv
HIP05551
Before First Filter (192489, 5)
After First Filter (163327, 5)
After Second Filter (160958, 5)
/home/vova/Documents/Projects/Receptor/emerson_2017/HIP00934.tsv
HIP00934
Before First Filter (281766, 5)
After First Filter (234112, 5)
After Second Filter (229698, 5)
/home/vova/Documents/Projects/Receptor/emerson_2017/HIP13625.tsv
HIP13625
Before First Filter (129090, 5)
After First Filter (108707, 5)
After Second Filter (106139, 5)
/home/vova/Documents/Projects/Receptor/emerson_2017/HIP12099.tsv
HIP12099
Before First Filter (252596, 5)
After First Filter (209179, 5)
After Second Filter (205539, 5)
/home/vova/Documents/Projects/Receptor/emerson_2017/HIP09150

/home/vova/Documents/Projects/Receptor/emerson_2017/HIP03630.tsv
HIP03630
Before First Filter (214220, 5)
After First Filter (178833, 5)
After Second Filter (175129, 5)
/home/vova/Documents/Projects/Receptor/emerson_2017/HIP13251.tsv
HIP13251
Before First Filter (164614, 5)
After First Filter (141794, 5)
After Second Filter (139804, 5)
/home/vova/Documents/Projects/Receptor/emerson_2017/HIP00904.tsv
HIP00904
Before First Filter (230938, 5)
After First Filter (186726, 5)
After Second Filter (182202, 5)
/home/vova/Documents/Projects/Receptor/emerson_2017/HIP13610.tsv
HIP13610
Before First Filter (232322, 5)
After First Filter (190486, 5)
After Second Filter (186893, 5)
/home/vova/Documents/Projects/Receptor/emerson_2017/HIP00898.tsv
HIP00898
Before First Filter (332224, 5)
After First Filter (270105, 5)
After Second Filter (265078, 5)
/home/vova/Documents/Projects/Receptor/emerson_2017/HIP14009.tsv
HIP14009
Before First Filter (197832, 5)
After First Filter (162871, 5)
After Second Filte

/home/vova/Documents/Projects/Receptor/emerson_2017/HIP03746.tsv
HIP03746
Before First Filter (298096, 5)
After First Filter (245823, 5)
After Second Filter (241056, 5)
/home/vova/Documents/Projects/Receptor/emerson_2017/HIP14024.tsv
HIP14024
Before First Filter (60835, 5)
After First Filter (51659, 5)
After Second Filter (50846, 5)
/home/vova/Documents/Projects/Receptor/emerson_2017/HIP01465.tsv
HIP01465
Before First Filter (215348, 5)
After First Filter (175777, 5)
After Second Filter (172333, 5)
/home/vova/Documents/Projects/Receptor/emerson_2017/Keck0112_MC1.tsv
Keck0112_MC1
Before First Filter (230609, 5)
After First Filter (194637, 5)
After Second Filter (191329, 5)
/home/vova/Documents/Projects/Receptor/emerson_2017/HIP13230.tsv
HIP13230
Before First Filter (71609, 5)
After First Filter (59312, 5)
After Second Filter (57689, 5)
/home/vova/Documents/Projects/Receptor/emerson_2017/Keck0056_MC1.tsv
Keck0056_MC1
Before First Filter (252179, 5)
After First Filter (216822, 5)
After Se

/home/vova/Documents/Projects/Receptor/emerson_2017/HIP00838.tsv
HIP00838
Before First Filter (398700, 5)
After First Filter (321945, 5)
After Second Filter (315271, 5)
/home/vova/Documents/Projects/Receptor/emerson_2017/HIP00869.tsv
HIP00869
Before First Filter (367425, 5)
After First Filter (302381, 5)
After Second Filter (296068, 5)
/home/vova/Documents/Projects/Receptor/emerson_2017/HIP09253.tsv
HIP09253
Before First Filter (330239, 5)
After First Filter (274327, 5)
After Second Filter (269663, 5)
/home/vova/Documents/Projects/Receptor/emerson_2017/HIP08507.tsv
HIP08507
Before First Filter (300445, 5)
After First Filter (244402, 5)
After Second Filter (238871, 5)
/home/vova/Documents/Projects/Receptor/emerson_2017/HIP05390.tsv
HIP05390
Before First Filter (345175, 5)
After First Filter (274498, 5)
After Second Filter (267103, 5)
/home/vova/Documents/Projects/Receptor/emerson_2017/HIP08725.tsv
HIP08725
Before First Filter (209471, 5)
After First Filter (172177, 5)
After Second Filte